<h2> More on Tensorflow </h2> 

GPUS dramatically speed up computations by splitting computations into many smaller chunks and running them in parallel across many GPU threads. 

In [63]:
import tensorflow as tf
import numpy as np

In [64]:
a = np.array([[1,2,3],[4,5,6]])
a

array([[1, 2, 3],
       [4, 5, 6]])

In [65]:
a.dtype

dtype('int32')

In [66]:
b = tf.Variable(a)
b

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [67]:
b.scatter_nd_update(indices = [[1,1]],updates = [100])
b

<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[  1,   2,   3],
       [  4, 100,   6]])>

In [68]:
#creating a simple tensor
t = tf.constant([[1,2,3],[4,5,6]])
t.shape

TensorShape([2, 3])

In [69]:
#indexing
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [5, 6]])>

In [70]:
tf.transpose(t)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 4],
       [2, 5],
       [3, 6]])>

In [71]:
tf.matmul(t,tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[14, 32],
       [32, 77]])>

<h3> Tensors and Numpy </h3> 

In [72]:
import numpy as np
a = np.array([2,4,5])

In [73]:
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 4, 5])>

In [74]:
#casting from 32bit float to a 64 bit float to sum in TF
t2 = tf.constant(40.,dtype = tf.float64)
t2

<tf.Tensor: shape=(), dtype=float64, numpy=40.0>

In [75]:
tf.constant(2.0) + tf.cast(t2,tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

<h3> Variables in Tensorflow </h3> 

We cannot modify constant tensors, however we can modify tf.Variable. These are required as we need to tweak weights in a neural network. 

In [76]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

A tf.Variable acts like a constant tensor and allows to perform the same operations. But it also allows you to modify th variable using the assign() method.

In [77]:
a = tf.constant([[1,2,3],[4,5,6]])
a

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]])>

In [78]:
v.assign(2*v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [79]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [80]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates = [100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [  8.,  10., 200.]], dtype=float32)>

In [81]:
v.scatter_nd_add(indices=[[0,0],[1,2]],updates = [100.,200.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[200.,  42.,   6.],
       [  8.,  10., 400.]], dtype=float32)>

scatter methods allows you to specify the index position and add to it

<h3> Sparse Tensors </h3> 

In a Sparse Tensor, you always need to give the positions first, then the values, then the dense shape

In [82]:
s = tf.SparseTensor(indices = [[0,1],[1,0],[2,3]],
                    values = [1.,2.,3.], 
                    dense_shape=[3,4])

In [83]:
tf.sparse.to_dense(s)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 0., 0.],
       [2., 0., 0., 0.],
       [0., 0., 0., 3.]], dtype=float32)>

Sparse tensors need to always be given in order of indices

In [84]:
s5 = tf.SparseTensor(indices=[[0, 1], [0, 2]],
                     values=[1., 2.],
                     dense_shape=[3, 4])

In [85]:
tf.sparse.to_dense(s5)

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 1., 2., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

<h3> Tensor Arrays </h3>

We first crrate the tensor with a fixed size. we cannot add past this

In [86]:
array = tf.TensorArray(dtype = tf.float32,size = 3)
array = array.write(0,tf.constant([1.,2.]))
array = array.write(1,tf.constant([3.,10.]))
array = array.write(2,tf.constant([5.,7.]))

<h3> Customizing models and training algorithms </h3> 

<h3> Custom Loss Functions </h3> 

In [87]:
#where y_true - y_pred absolute is less than 1, we want the squared loss and linear loss
def huber_fn(y_true,y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error)/2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error,squared_loss,linear_loss)

#we need to always use tensorflow functions to be used as custom loss functions. 

For the next step, we just apply this to our keras model

model.compile(loss = huber_fn, optimizer = "adam")

model.fit(.....)

In [88]:
#current implementation allows us to have one threshold of 1 in the Huber Function
#How to change the threshold?
def create_huber(threshold = 1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_errors = tf.abs(error) <  threshold
        #mae loss
        squared_loss = tf.square(error)/2
        #mse loss
        linear_loss = threshold*tf.abs(error) - threshold**2/2
        #if your error is less than threshold, return squared loss, 
        #else return linear_loss
        return tf.where(is_small_errors, squared_loss,linear_loss)
    return huber_fn

model.compile(loss = create_huber(2.0), optimizer = "adam")

The above causes an issue when saving the model using 
keras.callbacks.ModelCheckpoint("modelname.h5", save_best_only = True)

When we load the model again we will have to specify which function is being called for the loss function as the loss function is a custom one here

model = keras.models.load_model("modelname.h5", custom_objects = {"huber_fn":create_huber(2.0)})

In [89]:
from tensorflow import keras

In [90]:
#to avoid the above we can inherit the Keras.losses.loss method

class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold = 1.0, **kwargs):
        self.threshold = threshold
        #instantiate the superclass
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error)/2
        linear_loss = self.threshold*tf.abs(error) - self.threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        #you get the base configuration of superclass
        base_config = super().get_config()
        #add in the threshold and creates a new dictionary to be returned
        return {**base_config, "threshold":self.threshold}

get_config method returns a dictionary mapping each hyperparameter name to its value. It first calls the parent class's get_config() method, then adds the new hyperparameters to this dictionary.

from the above, when loading a model, we do this:

**model = keras.models.load_model("my_model_with_a_custom_loss_class.h5",
custom_objects={"HuberLoss": HuberLoss})**

Now we dont have to provide the threshold value too.

When we save the mode, Keras calls the loss instance (HuberLoss) get_config() method and the returned dictionary is stored as a JSON in the h5 file. 

When loaded, it calls **from_config()** and creates the instance of the class, passing the return from **from_config()** to **kwargs.

<h3>Custom Activation Functions, Initializers, Regularizers, and Constraints</h3>

In [91]:
#defining custom activation functions 
def my_softplus(z):
    return tf.math.log(tf.exp(z) + 1.0)

In [92]:
def my_glorot_initializer(shape,dtype = tf.float32):
    stddev = tf.sqrt(2/(shape[0] + shape[1]))
    return tf.random.normal(shape, stddev = stddev, dtype = dtype)

In [93]:
def my_li_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

In [94]:
#function to return only positive weights
def my_positive_weights(weights):
    return tf.where(weights<0, tf.zeros_like(weights),weights)

In [95]:
#custom regularizer using subclassing

class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,regfactor):
        self.factor = regfactor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"regfactor":self.factor}

For losses, layers and models we implement the call() method, and for regularizers, initializers and constraints we use the _____call_____() method. 

<h3>Custom Metrics </h3> 



In [96]:
a = tf.Variable([[2,2,2],[2,2,2]])
a = tf.cast(a,tf.float32)
a

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[2., 2., 2.],
       [2., 2., 2.]], dtype=float32)>

In [97]:
tf.reduce_sum(tf.abs(0.01*a))

<tf.Tensor: shape=(), dtype=float32, numpy=0.11999999>

At each training step the weights will be passed to the regularization function to compute the regularization loss. The return is then added to the main loss to get the final loss used for training. 

<h3> Custom Metrics </h3> 

Usually metrics keep track of the mean of a metric from each epoch. 

Suppose we have 5 true predictions, but 4 true positives. Thats 0.8 precision. Next epoch, we have 3 true predictions with 0 true positives. Thats 0 precision, but if we use mean, it becomes 0.4 precision overall

But the actual one is 8 true predictions with 4 true positives in total. Thats 0.5 precision. Hence we need an object to track the true positives and an object to track the predictions. 


In [98]:
from tensorflow import keras
precision = keras.metrics.Precision()
precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [99]:
precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

Note above that the same precision object, under 2 runs prduces the overal precision of the data fed into it. Not a mean, or not a new precision for the the new run.

This is called a streaming metric. 

In [100]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [101]:
def create_huber(threshold = 1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_errors = tf.abs(error) <  threshold
        #mae loss
        squared_loss = tf.square(error)/2
        #mse loss
        linear_loss = threshold*tf.abs(error) - threshold**2/2
        #if your error is less than threshold, return squared loss, 
        #else return linear_loss
        return tf.where(is_small_errors, squared_loss,linear_loss)
    return huber_fn

In [102]:
#creating streaming metrics
class HuberMetric(keras.metrics.Metric):
    def __init__(self,threshold = 1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        #returns a huber function to be used in this class
        self.huber_fn = create_huber(threshold)
        #add weight creates variables to keep track of attributes
        self.total = self.add_weight("total",initializer= "zeros")
        self.count = self.add_weight("count",initializer= "zeros")
    def update_state(self,y_true,y_pred,sample_weight = None):
        result = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(result))
        self.count.assign_add(tf.cast(tf.size(y_pred), tf.float32))
    def result(self):
        return self.total/self.count
    def get_config(self):
        base_config = self.get_config()
        return {**base_config, "threshold":self.threshold}

Both customPrecision and HuberMetric are inherriting from keras.metrics.Metric which is an abstract class. Abstract classes contain methods which need to be surely built in the child class.

In [103]:
class customPrecision(keras.metrics.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def update_state(self,ytrue, ypred,sample_weight = None):
        try: 
            for _class in self.unique_classes:
                self.total_true[_class] = self.total_true[_class] + tf.where(ytrue == _class).shape[0]
                self.total_prediction[_class] = self.total_prediction[_class] + tf.where(ypred == _class).shape[0]
                self.total_truepositives[_class] = self.total_truepositives[_class] + len([i for i in tf.where(ypred == _class) if i in tf.where(ytrue == _class)])
            #return [self.total_prediction,self.total_truepositives]
        except:
            self.unique,_,_ = tf.unique_with_counts(ytrue)
            #obtains the unique classes to a list that we can loop through
            self.unique_classes = self.unique.numpy()
            self.total_truepositives = {_class:0 for _class in self.unique_classes}
            self.total_prediction = {_class:0 for _class in self.unique_classes}
            self.precision_classbase = {_class:0 for _class in self.unique_classes}
            self.total_true = {_class:0 for _class in self.unique_classes}
            for _class in self.unique_classes:
                self.total_true[_class] = self.total_true[_class] + tf.where(ytrue == _class).shape[0]
                self.total_prediction[_class] = self.total_prediction[_class] + tf.where(ypred == _class).shape[0]
                self.total_truepositives[_class] = self.total_truepositives[_class] + len([i for i in tf.where(ypred == _class) if i in tf.where(ytrue == _class)])
            #return [self.total_prediction,self.total_truepositives]
    def result(self):
        for _class in self.unique_classes:
            self.precision_classbase[_class] = self.total_truepositives[_class]/self.total_prediction[_class]
            self.precision_classbase[_class] = self.precision_classbase[_class] * (self.total_true[_class]/sum(self.total_true.values()))
        return sum(self.precision_classbase.values())
    def get_config(self):
        base_config = self.get_config()
        return {**base_config}       

In [104]:
ytrue1 = tf.Variable([0, 1, 1, 1, 0, 1, 0, 1])
ypred1 = tf.Variable([1, 1, 0, 1, 0, 1, 0, 1])

In [105]:
prec_test1 = customPrecision()

In [106]:
prec_test1(ytrue1,ypred1)

<tf.Tensor: shape=(), dtype=float32, numpy=0.75>

In [107]:
prec_test1.precision_classbase

{0: 0.25, 1: 0.5}

In [108]:
ytrue2 = tf.Variable([0, 1, 0, 0, 1, 0, 1, 1])
ypred2 = tf.Variable([1, 0, 1, 1, 0, 0, 0, 0])

In [109]:
prec_test1(ytrue2,ypred2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4453125>

In [110]:
a = {0:4,1:3}

In [111]:
prec_test1.total_true

{0: 7, 1: 9}

<h3> Custom Layers </h3>

Sometimes we need to make layers which Tensorflow does not provide  default implementation for. In this case we need to create a custom layer.

Also we might want to club repeating layer patterns into one. 
E.g.: If you have layer pattern A,B,C,A,B,C,A,B,C we can make one layer D containing A,B,C and make D,D,D.

In [112]:
#to build custom layers with weights in them, we need to subclass of the keras.layers.Layer class
class MyDense(keras.layers.Layer):
    def __init__(self,units,activation = None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        #in build you build your kernel (weight matrix) and bias
    def build(self,batch_input_shape):
        #batch input shape is the number of features in the input to this layer
        #batch_input_shape = [batch_size, no.of features in each batch instance]
        self.kernel = self.add_weight(name = "kernel",
        shape = [batch_input_shape[-1],self.units],
        initializer="glorot_normal")
        self.bias = self.add_weight(name = "bias",
        shape = [self.units], initializer = "zeros")
        #sets self.built = True in parent class
        super().build(batch_input_shape)
    #call returns the output of the matrix multiplication or any result
    def call(self,X):
        return self.activation(tf.matmul(X,self.kernel) + self.bias)
    def compute_output_shape(self,batch_input_shape):
        #output shape will be [batch_size, no. of units]
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"units":self.units,
        "activation":keras.activations.serialize(self.activation)}

In [113]:
#using the above custom layer
model = keras.models.Sequential()
#notice that input_shape is part of the **kwargs argument
model.add(MyDense(30, activation = "relu", input_shape = (8,)))
model.add(MyDense(1))

In [114]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_dense_2 (MyDense)         (None, 30)                270       
_________________________________________________________________
my_dense_3 (MyDense)         (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [115]:
model.layers[0].bias

<tf.Variable 'my_dense_2/bias:0' shape=(30,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

<h3> Custom Models </h3> 

Lets say we are building a model with one dense input, which passes through an identical block(Residual Block), containing 2 dense layers and an addition of the input layer to the result of the 2 dense layers. The identical block is used 3 times.

In [116]:
#first we build this Residual Block
class ResidualBlock(keras.layers.Layer):
    def __init__(self,n_neurons,n_layers,**kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons,activation = "elu",kernel_initializer="he_normal") for _ in range(n_layers)]
    #call returns the output of the matrix multiplication or any result
    def call(self,inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        #as the Residual block adds the output of the layers into
        return inputs + Z

In [117]:
#notice that although we inherit from keras.layers.Layer, we can still use normal keras.layers.Dense in it, and implement calculations using these Dense layers from call()
class ResidualRegressor(keras.models.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30,activation = "elu",kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2,30)
        self.block2 = ResidualBlock(2,30)
        self.out = keras.layers.Dense(output_dim)
    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)
#we can use keras.layers.Layer to create custom Layers. we can use keras.models.Model to create custom models with custom layers in it. 

#then we can create instance of the keras.models.Model class and compile, fit as required

#Remember than keras.models.Model is a subclass of keras.layers.Layer with more functionality

In [118]:
model2 = ResidualRegressor(1)

In [119]:

block1 = ResidualBlock(2, 30)
model3 = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal"),
    block1, block1, block1, block1,
    ResidualBlock(2, 30),
    keras.layers.Dense(1)
])

<h3> Losses and Metrics Based on Model Internals </h3> 

Usually losses and metrics are based on predictions and ocassionaly sample weights. However we sometimes want it to be based on other parts of the model. 

In [120]:
class ReconstructingInputRegressor(keras.models.Model):
    def __init__(self,output_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation = "selu", kernel_initializer = "lecun_normal") for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
    def build(self,batch_input_shape):
        self.reconstructlayer = keras.layers.Dense(batch_input_shape[-1])
        self.build(batch_input_shape)
    def call(self,inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruct = self.reconstructlayer(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruct - inputs))
        self.add_loss(0.05*recon_loss)
        return self.out(Z)

In [153]:
class ReconstructingInputRegressor2(keras.models.Model):
    def __init__(self,output_dim,input_dim,**kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation = "selu", kernel_initializer = "lecun_normal") for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        self.reconstructlayer = keras.layers.Dense(input_dim)
        #adding a loss as a metric
        #self.reconmetric = keras.metrics.Mean(name = "recon_error")
    """ def build(self,batch_input_shape):
        self.reconstructlayer = keras.layers.Dense(batch_input_shape[-1])
        self.build(batch_input_shape) """
    def call(self,inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruct = self.reconstructlayer(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruct - inputs))
        #self.add_loss(0.05*recon_loss)
        self.add_metric(0.05*recon_loss, name = "recon_error")
        return self.out(Z)

In [154]:
import numpy as np
X_dummy = np.random.randn(10,8)
y_dummy = np.random.randn(10,1)

In [155]:
model = ReconstructingInputRegressor2(1,8)
model.compile(loss = "mse",optimizer = "adam")

In [156]:
model.fit(X_dummy,y_dummy, epochs = 3)

Epoch 1/3
1/1 [==============================] - 1s 710ms/step - loss: 2.8586 - recon_error: 0.0774
Epoch 2/3
1/1 [==============================] - 0s 4ms/step - loss: 2.0684 - recon_error: 0.0777
Epoch 3/3
1/1 [==============================] - 0s 6ms/step - loss: 1.4857 - recon_error: 0.0780
